### Deploy Result Analysis

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# data file parameter
file = "200 Classification"  # must be excel file
sheets = "Sheet3"  # "Sheet3"
start_first_column = 0  # start column location number

# classification for Part 1
threshold_num = 1

# word all usage in sent
word_use_num_max = 3
word_use_num_min = 1

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [756]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

In [ ]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/\
6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_youtube_sent_result.sort_values(by="search_string", key=lambda x:x.str.len(), ascending=True, inplace=True)
df_youtube_sent_result.drop(["word_index"], axis=1, inplace=True)
#df_youtube_sent_result = df_youtube_sent_result.sample(107)
df_youtube_sent_result

In [ ]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

In [ ]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

In [ ]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

In [ ]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [ ]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

In [ ]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index_list"])
df_youtube_sent_sample

In [ ]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list 

In [ ]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

In [ ]:
len(not_in_sent_word)

In [ ]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_word, df_youtube_sent_select, "search_string", 1, simple=False)
not_in_sent_selected_word_list_result = not_in_sent_selected_word_list_result.drop(["word"], axis=1)
not_in_sent_selected_word_list_result

In [ ]:
df_result_concat = pd.concat([df_youtube_sent_sample, not_in_sent_selected_word_list_result], axis=0)
df_result_concat.reset_index(drop=True, inplace=True)
df_result_concat

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
df_shared_word_manuel_count = word_count_result(df_result_concat, df_result_concat.columns[0:1])
df_shared_word_manuel_count.sort_values(by="word_count", inplace=True, ascending=False)
df_shared_word_manuel_count

In [ ]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [ ]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

In [ ]:
#df_result_concat.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)